In [ ]:
from huggingface_hub import list_datasets

all_datasets = [ds.id for ds in list_datasets()]
print(f"현재 허브에는 {len(all_datasets)}개의 데이터셋이 있습니다.")
print(f"처음 10개 데이터셋: {all_datasets[:10]}")

In [ ]:
from datasets import load_dataset

# emotion 데이터셋이 다운로드되지 않으면 SetFit/emotion을 사용합니다.
emotions = load_dataset("emotion")

In [ ]:
emotions

In [ ]:
train_ds = emotions["train"]
print(train_ds)

In [ ]:
len(train_ds)

In [ ]:
train_ds[0]

In [ ]:
train_ds.column_names

In [ ]:
print(train_ds.features)

In [ ]:
print(train_ds[:5])

In [ ]:
print(train_ds["text"][:5])

허깅페이스 허브에 필요한 데이터셋이 없을 경우

- csv => load_dataset("csv", data_files="my_file.csv")
- text => load_dataset("text", data_files="my_file.txt")
- json => load_dataset("json", data_files="my_file.jsonl")

load_dataset() 함수에 연관된 로딩 스크립트를 전달하고, data_files 매개변수에 팡리 경로나 URL을 하나 이상 지정한다.

2.1.2 데이터셋 => 데이터프레임

In [ ]:
import pandas as pd
emotions.set_format(type="pandas")
df = emotions["train"][:]
df.head()

In [ ]:
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

2.1.3 클래스 분포 살펴보기

In [ ]:
import matplotlib.pyplot as plt
df["label_name"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

불균형한 데이터 다루기

- 소수 클래스를 랜덤하게 오버샘플링
- 다수 클래스를 랜덤하게 언더샘플링
- 클래스의 대표성이 부족하다면, 레이블된 데이터를 더 많이 수집

2.1.4 트윗 길이 확인

In [ ]:
df["Words Per Tweet"] = df["text"].str.split().apply(len)
df.boxplot("Words Per Tweet", by="label_name", grid=False, showfliers=False, color="black")

plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
emotions.reset_format()

2.2 텍스트에서 토큰으로

2.2.1 문자 토큰화
- 가장 간단한 토큰화 방법은 각 문자를 개별로 모델에 주입하는 것이다.

In [ ]:
text = "Tokenizing text is a core task of NLP."
tokenized_text = list(text)
print(tokenized_text)

In [ ]:
token2idx = {
    ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))
}

print(token2idx)

In [ ]:
input_ids = [
    token2idx[token] for token in tokenized_text
]
print(input_ids)

input_idx를 원-핫 벡토의 2D 텐서로 변경한다.

In [ ]:
categorical_df = pd.DataFrame(
    {
        "Name": ["Bumblebee", "Optimus Prime", "Megatron"],
        "Label ID": [0 , 1, 2]
    }
)
print(categorical_df)

In [ ]:
pd.get_dummies(categorical_df["Name"])

In [ ]:
import torch
import torch.nn.functional as F

input_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))
print(one_hot_encodings.shape)

In [ ]:
print(f"토큰: {tokenized_text[0]}")
print(f"텐서 인덱스: {input_ids[0]}")
print(f"원-핫 인코딩: {one_hot_encodings[0]}")

2.2.2 단어 토큰화
- 텍스트의 일부 구조가 유지되는 토큰화 방법

In [ ]:
tokenized_text = text.split()
print(tokenized_text)

2.2.3 부분단어 토큰화
- 문자 토큰화와 단어 토큰화의 절충

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
from transformers import DistilBertTokenizer

distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)

In [ ]:
encoded_text = tokenizer(text)
print(encoded_text)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

In [ ]:
print(tokenizer.convert_tokens_to_string(tokens))

In [ ]:
print(tokenizer.vocab_size)

In [ ]:
print(tokenizer.model_max_length)

In [ ]:
print(tokenizer.model_input_names)

2.2.4 전체 데이터셋 토큰화하기

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
print(tokenize(emotions["train"][:2]))

In [ ]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

In [ ]:
print(emotions_encoded["train"].column_names)

2.3 텍스트 분류 모델 훈련하기

2.3.1 트랜스포머를 특성 추출기로 사용하기

In [ ]:
from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=AutoModel.from_pretrained(model_ckpt).to(device)

마지막 은닉 상태 추출하기

In [ ]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
print(f"입력 텐서 크기: {inputs['input_ids'].size()}") # 입력 텐서의 크기는 [batch_size, n_tokens]

In [ ]:
inputs = {
    k:v.to(device) for k, v in inputs.items()
}
with torch.no_grad(): # 그레이디턴트 자동 계산 비활성화를 위해 torch.no_grad()를 사용
    outputs = model(**inputs)
print(outputs)

In [ ]:
print(outputs.last_hidden_state.size()) # [batch_size, n_tokens, hidden_dim]

In [ ]:
print(outputs.last_hidden_state[:, 0].size())

In [ ]:
def extract_hidden_states(batch):
    # 모델 입력을 GPU로 옮긴다.
    inputs = {
        k: v.to(device) for k, v in batch.items()
        if k in tokenizer.model_input_names
    }
    # 마지막 은닉 상태를 추출한다.
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # [CLS] 토큰에 대한 벡터를 반환한다.
        return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [ ]:
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)

In [ ]:
print(emotions_hidden["train"].column_names)

특성 행렬 만들기

In [ ]:
import numpy as np

X_train = np.array(emotions_hidden["train"]["hidden_state"])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
print(f"X_train.shape: {X_train.shape} X_valid.shape: {X_valid.shape}")

훈련 세트 시각화

In [ ]:
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler

# 특성 스케일을 [0, 1] 범위로 조정한다.
X_scaled = MinMaxScaler().fit_transform(X_train)
# UMAP 객체를 생성하고 훈련한다.
mapper= UMAP(n_components=2, metric="cosine").fit(X_scaled)

# 2D 임베딩의 데이터프레임을 만든다.
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb["label"]=y_train
df_emb.head()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(7, 5))
axes=axes.flatten()
cmaps=["Greys", "Blues", "Oranges", "Reds", "Purples", "Greens"]
labels = emotions["train"].features["label"].names

for i, (label, cmap) in enumerate(zip(labels, cmaps)):
    df_emb_sub = df_emb.query(f"label=={i}")
    axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap, gridsize=20, linewidths=(0,))
    axes[i].set_title(label)
    axes[i].set_xticks([]), axes[i].set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
# 수렴을 보장하기 위해 max_iter를 증가시킨다
lr_clf=LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)
lr_clf.score(X_valid, y_valid)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_valid, y_valid)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

y_preds = lr_clf.predict(X_valid)
plot_confusion_matrix(y_preds, y_valid, labels)

2.3.2 트랜스포머 미세 튜닝

사전 훈련된 모델 로드

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 6
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

성공 지표 정의하기

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def comupte_metrics(pred):
    labels=pred.label_ids
    preds=pred.predictions.argmax(-1)
    f1=f1_score(labels, preds, average="weighted")
    acc=accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1
    }

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  log_level="error")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=emotions_encoded["train"],
    eval_dataset=emotions_encoded["validation"],
    tokenizer=tokenizer
)
trainer.train()

In [ ]:
preds_output=trainer.predict(emotions_encoded["validation"])

In [ ]:
preds_output.metrics

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
plot_confusion_matrix(y_preds, y_valid, labels)

오류 분석

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # 모든 입력 텐서를 모델과 같은 장치로 이동시킨다.
    inputs={
        k:v.to(device) for k, v in batch.items()
        if k in tokenizer.model_input_names
    }

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device), reduction="none")
    # 다른 데이터셋 열과 호환되도록 출력을 CPU로 옮긴다.
    return {
        "loss": loss.cpu().numpy(),
        "predicted_label": pred_label.cpu().numpy()
    }

In [ ]:
# 데이터셋을 다시 파이토치 텐서로 변환한다.
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 손실 값을 계산한다.
emotions_encoded["validation"]=emotions_encoded["validation"].map(
    forward_pass_with_label, batched=True, batch_size=16
)

텍스트, 손실, 예측 레이블과 진짜 레이블로 DataFrame을 만든다.

In [ ]:
emotions_encoded.set_format("pandas")
cols=["text", "label", "predicted_label", "loss"]
df_test=emotions_encoded["validation"][:][cols]
df_test["label"]=df_test["label"].apply(label_int2str)
df_test["predicted_label"]=(df_test["predicted_label"].apply(label_int2str))

In [ ]:
df_test.sort_values("loss", ascending=False).head(10)

In [ ]:
df_test.sort_values("loss", ascending=True).head(10)

In [ ]:
trainer.push_to_hub(commit_message="Training completed!")

In [ ]:
from transformers import pipeline

model_id = "kamja/distilbert-base-uncased-finetuned-emotion"
classifier = pipeline("text-classification", model=model_id)

In [ ]:
custom_tweet = "I saw a movie today and it was really good."
preds=classifier(custom_tweet, return_all_scores=True)

In [ ]:
preds_df = pd.DataFrame(preds[0])
plt.bar(labels, 100 * preds_df["score"], color="C0")
plt.title(f"'{custom_tweet}'")
plt.ylabel("Class probability (%)")
plt.show()